# Machine learning algorithms for coral bleaching classification 

## Load dataset

In [ ]:
'''
    Import libraries
'''
from sklearn import datasets
import pandas as pd
import numpy as np
from numpy import mean
from numpy import std
import sklearn
import seaborn as sb
import matplotlib.pyplot as plt
from numpy import mean 
from numpy import std
import pingouin as pg   
from scipy.stats import shapiro
from scipy.stats import levene 
from scipy.stats import bartlett
from scipy.stats import kruskal
import scikit_posthocs as sp 
from scipy import stats
from sklearn.datasets import make_classification
from sklearn.metrics import confusion_matrix, classification_report
from sklearn import metrics
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import RepeatedKFold
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import log_loss
from sklearn.metrics import roc_auc_score
from itertools import combinations, permutations
# check scikit-learn version
print(sklearn.__version__)

In [ ]:
'''
    Load full dataset
'''
data = pd.read_csv('df_sst_clouds.csv',low_memory=False)
len(data)  

In [ ]:
list(data.columns)

In [ ]:
'''
    Subset DF by SEVERITY_CODE [0,1,2,3]
      Severity_code == "-1" is dropped
'''
#data = data.dropna() # drop rows that contains NaN's 
data = data[(data.SEVERITY_CODE == 0)|(data.SEVERITY_CODE == 1)|(data.SEVERITY_CODE == 2)|(data.SEVERITY_CODE == 3)] 
#data = data[(data.YEAR >= 2002)] # First year with more than 100 records
#data = data[(data.YEAR >= 2015) & (data.YEAR <= 2016)] # subset a single event
#data = data[(data.YEAR >= 1997) & (data.YEAR <= 1998)]
#list(data.columns)
data = data.dropna() # drop rows that contains NaN's (if any)
len(data)

In [ ]:
'''
            Restricting to cells with an actual DHW value only
    This step should be used after getting results of complegte dataset
'''
# data = data.drop(data[(data.DHW == 0) & (data.DHW_RMax30 == 0) & (data.DHW_RMax60 == 0) &(data.DHW_RMax90 == 0) & (data.SEVERITY_CODE > 0)].index) 
# len(data)

In [ ]:
'''
        Create dataframes to build independent models 
        Example:
        X1 = data[['var #1','var #2']]  ## features (dependen variable(s))
        X2 = data[['var #1','var #3']] 
        y = data[['independent variable']]  ## labels (indipendent variable)
        The object "X_" can be used to run a specific model, however, to keep it simple, the object "X" is going to be used and edited each time to get printed the results and avoid changing the code in subsequent steps" 
'''
# ## Features (dependent variable)
# X0=data[['DHW']] # better than any othe DHW metric
# X00=data[['DHW_9']] # better than DHW
# X1=data[['DHW', 'CF']] 
# X2=data[['DHW', 'CFrunmean7']]
# X3=data[['DHW', 'CFrunmean30']] 
# X4=data[['DHW', 'CFrunmean90']] # 
# X5=data[['DHW', 'CF_a']]
# X6=data[['DHW', 'CF_a_runmean7']] #
# X7=data[['DHW', 'CF_a_runmean30']] #
# X8=data[['DHW', 'CF_a_runmean90']] # +
# X9=data[['DHW_9','CF_a']] # + better than DHW_9
# X10=data[['DHW_9','CFrunmean7']]
# X11=data[['DHW_9','CFrunmean30']]
# X12=data[['DHW_9','CFrunmean90']]
# X13=data[['DHW_9','CF_a_runmean7']] # 
# X12=data[['DHW_9','CF_a_runmean7']] # +
# X15=data[['DHW_9','CF_a_runmean90']] # 
# X16=data[['DHW_9','CFrunmean7','CD']] # x
# X17=data[['DHW_9','CFrunmean7','WD']] # x
# X18=data[['SSTrunmax90','CF_a_runmean90','WD']] # +++
# X19=data[['DHW','CF_a_runmean7','CV_run7']] # +
# X20=data[['DHW','CF_a_runmean30','CV_run30']] # +
# X21=data[['DHW','CF_a_runmean90','CV_run90']] # -
# X22=data[['SSTrunmean90','CF_a_runmean90','WD']] # +
# X23=data[['SSTrunmean7','CF_a_runmean90','WD']]
# X24=data[['SSTrunmax7','CF_a_runmean90','WD']] # ++
# X25=data[['YEAR','DHW_9','CF_a_runmean7']] # +
# X26=data[['YEAR','DHW','CF_a_runmean90']] # +
# X27=data[['YEAR','CF_a_runmean90']] # -   VIF + 1,5
# X28=data[['YEAR','SSTrunmax7','CFrunmean90']] # 
# X30=data[['YEAR','SSTrunmax90','DHW','CV_run90']] # +++
# X31=data[['YEAR','SSTrunmax90','DHW','CV_run90','CFrunmean90']] # +++
# X32=data[['DHW_adj_date', 'CFrunmean90_adj_date']] # YEAR + DHW_9 VIF + 1.5
# #labels (indipendent variable)
# y=data['SEVERITY_CODE'] 

In [ ]:
X = data.loc[:,('DHW','CF_a_runmean90')] 

In [ ]:
'''
Iterable object to run the models 
Here you can set the model's desired variables
'''
X=data.loc[:,('DHW','CF_a_runmean30')] ## Features (dependent variable(s)); select desired variables
y=data['SEVERITY_CODE'] # labels (indipendent variable)

In [ ]:
'''
    Variance inflation factor VIF
'''
from statsmodels.stats.outliers_influence import variance_inflation_factor
# Get variables for which to compute VIF and add intercept term
X['Intercept'] = 1
# Compute and view VIF
vif = pd.DataFrame()
vif["variables"] = X.columns
vif["VIF"] = [variance_inflation_factor(X.values, i) for i in range(X.shape[1])]
# View results using print
print(vif)

# Random forest classifier

In [ ]:
'''
    Build the models
'''
model = RandomForestClassifier(n_estimators=200, random_state=3)
model.fit(X,y)
# evaluate the model
cv = RepeatedKFold(n_splits=5, n_repeats=10, random_state=3)
n_scores = cross_val_score(model, X, y, cv=cv) #n_jobs=-1, error_score='raise'

In [ ]:
'''
    Report performance
'''
print('Cross val score: %.3f (%.3f)' % (mean(n_scores), std(n_scores)))
print('\n')
# Features importance
print('=== features importances ===')
fi = pd.DataFrame({'feature': list(X.columns),
                   'importance': model.feature_importances_}).\
                    sort_values('importance', ascending = False)
fi

In [ ]:
'''
    Confusion matrix
'''
y_pred=model.predict(X)
conf_mat = confusion_matrix(y, y_pred)
conf_mat_norm = confusion_matrix(y, y_pred,normalize='all')
print("=== Confusion matrix ===")
print(conf_mat)
print('\n')
print("=== Confusion matrix normalized ===")
print(conf_mat_norm)
print('\n')
print("=== Classification Report ===")
print(classification_report(y, y_pred))
print('\n')
print('=== Accuracy and Kappa ===')
print('accuracy', metrics.accuracy_score(y, y_pred))
print('\n')
print('kappa', metrics.cohen_kappa_score(y, y_pred))
print('\n')

In [ ]:
'''
    Evaluation between classifications models through "log loss"
'''
model_probs = model.predict_proba(X)
score = log_loss(y, model_probs)

'''
    Evaluation between classifications models through "ROC_AUC"
'''
roc_value = roc_auc_score(y, model_probs, multi_class='ovo') # ovo': Computes the average AUC of all possible pairwise combinations of classes


print('=== roc_auc_score ===') 
print(roc_value)
print(' ')
print('=== log_loss_score ===') 
print(score)


In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt     
ax = plt.subplot()
#Heat map with annot=True to annotate cells
sns.heatmap(conf_mat, annot=True, ax = ax, fmt='d', cmap='Blues') # actual cases
#sns.heatmap(conf_mat/np.sum(conf_mat), annot=True, ax = ax, fmt='.2%', cmap='Blues') #percentage
# labels, title and ticks
ax.set_xlabel('Predicted bleaching level');ax.set_ylabel('True bleaching level'); 
ax.set_title('Confusion Matrix'); 
ax.xaxis.set_ticklabels(['level 0','level 1','level 2', 'level 3']); ax.yaxis.set_ticklabels(['level 0','level 1','level 2', 'level 3'])
#plt.show()
#plt.savefig('DHW9_CF_a_runmean90.pdf', dpi=300)

In [ ]:
comparisons = pd.DataFrame({'Real':y, 'Predictions':y_pred})
comparisons.to_csv('pred_DHW_CF_a_runeman90.csv')
print(comparisons[['Real','Predictions']])

In [ ]:
data.to_csv('df_for_RFoutputs.csv')